## I. FastAPI

### 1.1. Introduction

### 1.2. Operations

In [2]:
import pandas as pd
pd.DataFrame(
    {
        "GET": [0.1, 0.2, 0.3],
        "POST": [0.2, 0.3, 0.4],
        "PUT": [0.3, 0.4, 0.5],
        "DELETE": [0.4, 0.5, 0.6]
    },
    index=["Traditional use", "Parameter","ascbashkc"]
)

,GET,POST,PUT,DELETE
Traditional use,0.1,0.2,0.3,0.4
Parameter,0.2,0.3,0.4,0.5
ascbashkc,0.3,0.4,0.5,0.6


In [ ]:
from fastapi import FastAPI
from pydantic_basemodel import *

# Create an instance of FastAPI / Instantiate app as an object of the FastAPI class
app = FastAPI()

# Handle get requests to the root which is either the host alone or the host followed by a single slash
@app.get("/")
# Add a query parameter name with a default value "Alan".
async def root(name: str = "Alan"):
    # When it return, FastAPI will convert the dictionary to JSON and return it to the client which will be shown in the browser
    return {"message": f"Hello, {name}"}

# Handle post requests to the /reviews endpoint
@app.post("/reviews", response_model=DbReview)
def create_review(review: MovieReview):
    # Typically we would define a file call crud.py with custom functions to create, read, update, and delete objects in the database. 
    db_review = crud.create_review(review)
    return db_review

@app.put("/reviews", response_model=DbReview)
def update_review(review: MovieReview):
    # update the movie review in the database
    db_review = crud.update_review(review)
    # return the updated review
    return db_review

@app.delete("/reviews", response_model=DbReview)
def delete_review(review: MovieReview):
    # delete the movie review in the database
    db_review = crud.delete_review(review)
    # return the nothing since the data is gone
    return {}


### 1.3. Handling Errors

In [ ]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel

# Define model Item
class Item(BaseModel):
    name: str

# Define items at application startup
items = {"apples", "oranges"}

app = FastAPI()


@app.delete("/items")
def delete_item(item: Item):
    name = item.name
    if name in items:
        items.remove(name)
    else:
        # Raise HTTPException with status code for "not found"
        raise HTTPException(status_code=404, detail="Item not found.")
    return {}

fastapi dev main.py

curl -X DELETE \
  -H 'Content-Type: application/json' \
  -d '{"name": "bananas"}' \
  http://localhost:8000/items

### 1.4. Using async for concurrent work

#### 1.4.1. When to use async

- Use `async`
    + If our application does not have to communicate with anything else and wait for it to respond
    + e.g. Audio | Image Processing, Computer Vision, ML, DL, etc.
    + When all the functions within our endpoint can be called with `await`
- Don't use `async`
    + If our application has to communicate with
        - File system 
        - Database
        - Another server
    + If we are not sure

#### 1.4.2. Asynchronous DELETE operation

You've been asked to create an API endpoint that deletes items managed by your API. To accomplish this, create an endpoint `/items` that serves HTTP DELETE operations. Make the endpoint asynchronous, so that your application can continue to serve requests while maintaining any long-running deletion tasks.

We can't run the FastAPI server directly with "Run this file" - see the instructions for how to run the server and test your code from the terminal.

**Instructions**
- Make the delete operation asynchronous.
- Validate the existence of `item.name` in list `items`.
- Return the appropriate status code for "not found."
- Run the live server from the terminal: `fastapi dev main.py`.
- Open a new terminal and test your code with the following command:
```
curl -X DELETE \
  -H 'Content-Type: application/json' \
  -d '{"name": "rock"}' \
  http://localhost:8000/items

curl -X DELETE \
  -H 'Content-Type: application/json' \
  -d '{"name": "roll"}' \
  http://localhost:8000/items
```

In [ ]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel

# Define model Item
class Item(BaseModel):
    name: str

app = FastAPI()

items = {"rock", "paper", "scissors"}


@app.delete("/items")
# Make asynchronous
async def root(item: Item):
    name = item.name
    # Check if name is in items
    if name not in items:
        # Return the status code for not found
        raise HTTPException(status_code=404, detail="Item not found.")
    items.remove(name)
    return {"message": "Item deleted"}

You've created and tested your first asynchronous DELETE endpoint. This is the traditional operation for endpoints that delete data managed by the API, and the final operation you need to build an application with a CRUD (Create, Read, Update & Delete) API.

### 1.5. FastAPI automated testing
how to test our endpoints automatically

#### 1.5.1. Automated Tests

- **Unit Tests**
    + Focus: Isolated code
    + Purpose: Validate code function
    + Scope: Function or method
    + Environment: Isolated Python env with necessary dependencies 
    + Use `pytest`
```
def test_main():
    response = main()
    assert response == {"msg":"Hello"}
```

- **System Tests**
    + Focus: Isolated system operations
    + Purpose: Validate system function 
    + Scope: Endpoint
    + Environment: Python env with access to the running application 
    + Use `pytest` with `TestClient`
```
def test_read_main():
    response = client.get("/")
    assert response.status_code == 200
    assert response.json() == {"msg":"Hello"}
```

#### 1.5.2. System test

You've built your FastAPI application and added unit tests to verify code functionality. Writing a system test for an API endpoint will ensure that the endpoint works on the running application.

We can't run the FastAPI server directly with "Run this file" - see the instructions for how to run the server and test your code from the terminal.

##### Instructions

- Review the GET endpoint defined in `main.py`.
- Complete the following system test in `system_test.py`
- In the terminal, run `pytest`.

##### main.py

In [ ]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from typing import Optional

# define model Item
class Item(BaseModel):
    name: str
    quantity: Optional[int] = 0

app = FastAPI()

items = {"scissors": Item(name="scissors", quantity=100)}


@app.get("/items")
def read(name: str):
    if name not in items:
        raise HTTPException(status_code=404, detail="Item not found")
    return items[name]

##### system_test.py

In [ ]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from typing import Optional

# define model Item
class Item(BaseModel):
    name: str
    quantity: Optional[int] = 0

app = FastAPI()

items = {"scissors": Item(name="scissors", quantity=100)}


@app.get("/items")
def read(name: str):
    if name not in items:
        raise HTTPException(status_code=404, detail="Item not found")
    return items[name]

### 1.6. Building a JSON CRUD APIs

![](image/CRUD.jpg)

#### 1.6.1. JSON CRUD API Motivation

##### Fundamentals

- Manage the entire object lifecycle
- Understand best practices for HTTP API operations
- Design our own data management APIs

##### Opportunities

- Inject business logic for more complex data operations
- Build high throughput data
- Build ML inference pipelines

#### 1.6.2. Building a CRUD Module

##### Assume we have defined 2 pydantic models for a movie review

In [ ]:
from pydantic import BaseModel

class Review(BaseModel):
    movie: str
    num_stars: int
    text: str

class DbReview(Review):
    movie: str
    num_stars: int
    text: str
    # Reference database ID of Reviews
    review_id: int

##### CRUD Module: crud.py

In [ ]:
def create_review(review: Review):
    # Create a review in the database
    # Return the review with the database ID
    return DbReview(**review.dict(), review_id=1)

def read_review(review_id: int):
    # Read a review from the database
    # Return the review with the database ID
    return DbReview(movie="The Matrix", num_stars=5, text="Awesome movie!", review_id=review_id)

def update_review(review: DbReview):
    # Update a review in the database
    # Return the review with the database ID
    return review

def delete_review(review_id: int):
    # Delete a review from the database
    # Return the review with the database ID
    return DbReview(movie="The Matrix", num_stars=5, text="Awesome movie!", review_id=review_id)

#### Build a completed JSON CRUD APIs

In [ ]:
# main.py

from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from typing import Optional

# define model Item
class Item(BaseModel):
    name: str
    quantity: Optional[int] = 0

app = FastAPI()

items = {}


@app.post("/items")
def create(item: Item):
    name = item.name
    if name in items:
        raise HTTPException(status_code=409, detail="Item exists")
    items[name] = item
    return {"message": f"Added {name} to items."}
  
@app.get("/items")
def read(name: str):
    if name not in items:
        raise HTTPException(status_code=404, detail="Item not found")
    return items[name]  
  
@app.put("/items")
def update(item: Item):
    name = item.name
    if name not in items:
        raise HTTPException(status_code=404, detail="Item not found")
    items[name] = item
    return {"message": f"Updated {name}."}
  
@app.delete("/items")
def delete(item: Item):
    name = item.name
    if name not in items:
        raise HTTPException(status_code=404, detail="Item not found")
    del items[name]
    return {"message": f"Deleted {name}."}

#### Test

- Run the live server from the terminal: `fastapi dev main.py`
- Open a new terminal and test your code with the following five commands:
```
curl -X POST \
  -H 'Content-Type: application/json' \
  -d '{"name": "rock"}' \
  http://localhost:8000/items

curl http://localhost:8000/items?name=rock

curl -X PUT \
  -H 'Content-Type: application/json' \
  -d '{"name": "rock", "quantity": 100}' \
  http://localhost:8000/items

curl -X DELETE \
  -H 'Content-Type: application/json' \
  -d '{"name": "rock"}' \
  http://localhost:8000/items

curl http://localhost:8000/items?name=rock
```

## II. Django